In [1]:
# 读取数据
import pandas as pd
world_data = pd.read_csv('today_world_2020_04_03.csv')
import pyecharts
# 调整配置项
import pyecharts.options as opts
# Map类用于绘制地图
from pyecharts.charts import Map

In [2]:
#确定现存确诊
world_data['today_storeConfirm'] = world_data['total_confirm'] - world_data['total_heal'] - world_data['total_dead']
world_data.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,9577772,2020-04-03 07:02:37,突尼斯,455,0,3,14,0,NaN,32.0,0.0,0.0,2.0,0.0,438,NaN
1,9507896,2020-04-03 21:29:41,塞尔维亚,1476,0,0,89,0,NaN,305.0,0.0,0.0,58.0,0.0,1387,NaN
2,0,2020-04-03 22:02:41,中国,82858,135,76816,3331,379,873.0,134.0,12.0,206.0,4.0,0.0,2711,32.0
3,1,2020-04-03 11:25:22,日本,3834,0,1133,88,0,NaN,351.0,NaN,9.0,4.0,NaN,2613,NaN
4,2,2020-04-03 12:48:49,泰国,1978,0,581,19,0,NaN,103.0,0.0,76.0,4.0,0.0,1378,NaN


In [3]:
contry_name = pd.read_csv('county_china_english.csv', encoding='GB2312')
contry_name.head()

,英文,中文
0,Afghanistan,阿富汗
1,Aland Islands,奥兰群岛
2,Albania,阿尔巴尼亚
3,Algeria,阿尔及利亚
4,American Samoa,美属萨摩亚


In [4]:
#将数据世界数据的中文名换成英文
world_data['eg_name'] = world_data['name'].replace(contry_name['中文'].values ,contry_name['英文'].values)
world_data['eg_name'].head()

0     Tunisia
1      Serbia
2       China
3       Japan
4    Thailand
Name: eg_name, dtype: object

In [5]:
#将国家和现存人数转换成一个矩阵（嵌套列表）
heatmap_data = world_data[['eg_name','today_storeConfirm']].values.tolist()
heatmap_data[:10]

[['Tunisia', 438],
 ['Serbia', 1387],
 ['China', 2711],
 ['Japan', 2613],
 ['Thailand', 1378],
 ['Singapore', 843],
 ['Korea', 3867],
 ['Australia', 4653],
 ['Germany', 61241],
 ['United States', 230278]]

In [6]:
#开始绘图准备
map_ = Map().add(series_name = "现存确诊人数", # 设置提示框标签
                 data_pair = heatmap_data, # 输入数据
                 maptype = "world", # 设置地图类型为世界地图
                 is_map_symbol_show = False # 不显示标记点
                )
# 设置系列配置项
map_.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示国家（标签）名称
# 设置全局配置项
map_.set_global_opts(title_opts = opts.TitleOpts(title="世界各国家现存确诊人数地图"), # 设置图标题
                     # 设置视觉映射配置项
                     visualmap_opts = opts.VisualMapOpts(pieces=[ # 自定义分组的分点和颜色
                                                               {"min": 10000,"color":"#800000"}, # 栗色
                                                               {"min": 5000, "max": 9999, "color":"#B22222"}, # 耐火砖
                                                               {"min": 999, "max": 4999,"color":"#CD5C5C"}, # 印度红
                                                               {"min": 100, "max": 999, "color":"#BC8F8F"}, # 玫瑰棕色
                                                               {"max": 99, "color":"#FFE4E1"}, # 薄雾玫瑰
                                                              ],
                     is_piecewise = True))  # 显示分段式图例

# 在notebook中进行渲染(生成动态图)
map_.render_notebook()

开始绘制玫瑰图

In [7]:
#导入饼图（玫瑰图属于饼图类）绘制包
from pyecharts.charts import Pie

In [8]:
#首先筛选出累计死亡人数超过500人的世界国家，并按人数进行降序排序
need_data = world_data[['name','total_dead']][world_data['total_dead'] >500]
rank = need_data[['name','total_dead']].sort_values(by='total_dead',ascending=False).values
rank

array([['意大利', 13915],
       ['西班牙', 10935],
       ['美国', 6069],
       ['法国', 5398],
       ['英国', 3605],
       ['中国', 3331],
       ['伊朗', 3294],
       ['荷兰', 1487],
       ['德国', 1107],
       ['比利时', 1043]], dtype=object)

In [9]:
#基本配置信息
pie = Pie().add("累计死亡人数", # 添加提示框标签
                rank, # 输入数据
                radius = ["20%", "80%"],  # 设置内半径和外半径
                center = ["60%", "60%"],  # 设置圆心位置
                rosetype = "radius")   # 玫瑰图模式，通过半径区分数值大小，角度大小表示占比

In [10]:
#全局和系列配置信息
pie.set_global_opts(title_opts = opts.TitleOpts(title="世界国家累计死亡人数玫瑰图",  # 设置图标题
                                                pos_right = '40%'),  # 图标题的位置
                    legend_opts = opts.LegendOpts( # 设置图例
                                                orient='vertical', # 垂直放置图例
                                                pos_right="85%", # 设置图例位置
                                                pos_top="15%"))

pie.set_series_opts(label_opts = opts.LabelOpts(formatter="{b} : {d}%")) # 设置标签文字形式为（国家：占比（%））

In [11]:
# 在notebook中进行渲染
pie.render_notebook()

3月美国单日新增确诊人数与股票指数涨跌幅折线图

In [12]:
stock = pd.read_csv('stockindex.csv',encoding='GB2312')
stock

,日期,NASDAQ,SSEC,N225
0,2020-03-02,4.49,3.15,0.95
1,2020-03-03,-2.99,0.74,-1.22
2,2020-03-04,3.85,0.63,0.08
3,2020-03-05,-3.10,1.99,1.09
4,2020-03-06,-1.86,-1.21,-2.72
5,2020-03-09,-7.29,-3.01,-5.07
6,2020-03-10,4.95,1.82,0.85
7,2020-03-11,-4.70,-0.94,-2.27
8,2020-03-12,-9.43,-1.52,-4.41
9,2020-03-13,9.35,-1.23,-6.08


In [13]:
#读取全球疫情历史数据
alltime_data = pd.read_csv('alltime_world_2020_04_04.csv')
alltime_data

,date,total_confirm,total_suspect,total_heal,total_dead,total_severe,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,name
0,2020-03-03,1,0,0,0,0,1,0.0,0,0,NaN,NaN,突尼斯
1,2020-03-08,2,0,0,0,0,1,0.0,0,0,0.0,NaN,突尼斯
2,2020-03-09,5,0,0,0,0,3,0.0,0,0,0.0,NaN,突尼斯
3,2020-03-11,7,0,0,0,0,2,0.0,0,0,0.0,NaN,突尼斯
4,2020-03-12,13,0,0,0,0,6,0.0,0,0,0.0,NaN,突尼斯
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3607,2020-03-29,43,0,0,0,0,15,0.0,0,0,0.0,NaN,马达加斯加
3608,2020-03-30,46,0,0,0,0,3,0.0,0,0,0.0,NaN,马达加斯加
3609,2020-03-31,57,0,0,0,0,11,0.0,0,0,0.0,NaN,马达加斯加
3610,2020-04-02,59,0,0,0,0,2,0.0,0,0,0.0,NaN,马达加斯加


In [14]:
#筛选出与股票开盘日期对应的美国单日新增确诊人数：
import warnings
warnings.filterwarnings('ignore')
#选出美国的数据
alltime_us = alltime_data[alltime_data['name'] == '美国']
#选出美国在开盘日的确诊数据
use_data = alltime_us[alltime_data['date'].isin(stock['日期'].values)][['date','today_confirm']]
use_data

,date,today_confirm
428,2020-03-02,7
429,2020-03-03,26
430,2020-03-04,41
431,2020-03-05,69
432,2020-03-06,92
435,2020-03-09,170
436,2020-03-10,287
437,2020-03-11,308
438,2020-03-12,346
439,2020-03-13,546


In [15]:
#导入绘制折线图的Line类和绘制组合图形的Grid类：
from pyecharts.charts import Line, Grid

In [16]:
#定义美国单日新增确诊人数折线图的相关设置
l1 = Line().add_xaxis(# 配置x轴
                      xaxis_data = use_data['date'].values   # 输入x轴数据
                      )

l1.add_yaxis(# 配置y轴
             series_name = "单日新增人数",  # 设置图例名称
             y_axis = use_data['today_confirm'].values.tolist(),  # 输入y轴数据
             symbol_size = 10, # 设置点的大小
             label_opts = opts.LabelOpts(is_show=False), # 标签设置项：显示标签
             linestyle_opts = opts.LineStyleOpts(width=1.5, type_='dotted'), # 线条宽度和样式
             is_smooth = True, # 绘制平滑曲线
             )

# 设置全局配置项
l1.set_global_opts(title_opts = opts.TitleOpts(title = "3月美国单日新增人数与股票指数涨幅对比折线图",
                                               pos_left = "center"), # 设置图标题和位置
                   axispointer_opts = opts.AxisPointerOpts(is_show = True,
                                                           link = [{"xAxisIndex": "all"}]),  # 坐标轴指示器配置
                   # x轴配置项
                   xaxis_opts = opts.AxisOpts(type_ = "category",
                                              boundary_gap = True), # 坐标轴两边是否留白
                   # y轴配置项
                   yaxis_opts = opts.AxisOpts(name = "单日新增人数"), # 轴标题
                   # 图例配置项
                   legend_opts = opts.LegendOpts(pos_left  ='7%') # 图例的位置
                   )


In [17]:
#定义三支股票指数变化的折线图设置：
l2 = Line().add_xaxis(xaxis_data = use_data['date'].values)

l2.add_yaxis(series_name = "上证指数",
             y_axis = stock['SSEC'].values,  # 添加上证指数数据
             symbol_size = 10,
             label_opts = opts.LabelOpts(is_show = False),
             linestyle_opts = opts.LineStyleOpts(width = 1.5), # 设置线宽
             is_smooth = True)

l2.add_yaxis(series_name = "日经225指数",
             y_axis = stock['N225'].values,  # 添加日经225指数数据
             symbol_size = 10,
             label_opts = opts.LabelOpts(is_show = False),
             linestyle_opts = opts.LineStyleOpts(width = 1.5),
             is_smooth = True)

l2.add_yaxis(series_name = "纳斯达克综合指数",
             y_axis = stock['NASDAQ'].values, # 添加纳斯达克综合指数数据
             symbol_size = 10,
             label_opts = opts.LabelOpts(is_show = False),
             linestyle_opts = opts.LineStyleOpts(width = 1.5),
             is_smooth = True)

l2.set_global_opts(axispointer_opts = opts.AxisPointerOpts(  # 设置坐标轴指示器
                                                           is_show = True,
                                                           link = [{"xAxisIndex": "all"}]),  # 对x轴所有索引进行联动

                   xaxis_opts = opts.AxisOpts(grid_index = 1,  # x轴开始的索引
                                             type_ = "category", # 类型
                                             boundary_gap = True,
                                             position = "top", # 坐标轴位置
                                             axisline_opts = opts.AxisLineOpts(is_on_zero=True)),  # x轴或y轴的轴线是否在另一个轴的0刻度上

                   yaxis_opts = opts.AxisOpts(is_inverse = False, name = "涨跌幅（%）",name_gap = 25), # 轴线设置
                   legend_opts = opts.LegendOpts(pos_bottom = '50%',pos_right = '70') # 图例设置
                   )

In [18]:
#将两幅图按照上下位置进行组合：
# 绘制组合图形
grid = Grid(init_opts = opts.InitOpts(width = "1024px", height = "768px")) # 设置图形的长和宽

grid.add(chart=l1,  # 添加第一个图表
         grid_opts = opts.GridOpts(pos_left = 50, pos_right = 50, height = "35%"))  # 直角坐标系网格配置项

grid.add(chart = l2, # 添加第二个图表
         grid_opts = opts.GridOpts(pos_left = 50, pos_right = 50, pos_top = "55%", height = "35%"))

# 利用notebook进行渲染
grid.render_notebook()

3月世界国家累计确诊人数动态条形图

In [19]:
country_list = ['美国', '意大利', '中国', '西班牙', '德国', '伊朗', '法国', '英国', '瑞士','比利时']
need_data = alltime_data[alltime_data['name'].isin(country_list)]

接下来使用datetime模块生成时间数据，构造时间列表：

In [20]:
from datetime import datetime,timedelta
time_list = [(datetime(2020, 3, 1) + timedelta(i)).strftime('%Y-%m-%d') for i in range(31)]

In [21]:
#载入Matplotlib库，并设置正常显示中文字体：
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['figure.dpi'] = 100

In [22]:
#为每个国家设置一种颜色：
color_list = ['brown','peru','orange','blue','green',
              'red','yellow','teal','pink','orchid']
country_color = pd.DataFrame()
country_color['country'] = country_list
country_color['color'] = color_list

In [23]:
#定义绘图函数：
import matplotlib.ticker as ticker

def barh_draw(day):
    
    # 提取每一天的数据
    draw_data = need_data[need_data['date']==day][['name','total_confirm']].sort_values(by='total_confirm',ascending=True)
    
    # 清空当前的绘图
    ax.clear()
    
    # 绘制条形图
    ax.barh(draw_data['name'],draw_data['total_confirm'], color=[country_color[country_color['country']==i]['color'].values[0] for i in draw_data['name']])
    
    # 数值标签的间距
    dx = draw_data['total_confirm'].max()/200
    
    # 添加数值标签
    for j, (name, value) in enumerate(zip(draw_data['name'], draw_data['total_confirm'])):
        
        ax.text(value+dx, j, f'{value:,.0f}', size=10, ha='left', va='center')
        
    # 添加日期标签
    ax.text(draw_data['total_confirm'].max()*0.75, 0.4, day, color='#777777',size=40, ha='left')
    
    # 设置刻度标签的格式
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    
    # 设置刻度的位置
    ax.xaxis.set_ticks_position('top')

    # 设置刻度标签的颜色和大小
    ax.tick_params(axis='x',colors='#777777', labelsize=15)
    
    # 添加网格线
    ax.grid(which='major',axis='x',linestyle='-')
    
    # 添加图标题
    ax.text(0, 11, '3月世界各国家累计确诊人数动态条形图',size=20, ha='left')
    
    # 去除图边框
    plt.box(False)
    
    # 关闭绘图框
    plt.close()

In [24]:
# 动态绘图
fig, ax = plt.subplots(figsize=(12, 8))

import matplotlib.animation as animation
from IPython.display import HTML

animator = animation.FuncAnimation(fig, barh_draw, frames=time_list, interval=200)
HTML(animator.to_jshtml())

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
